## JuPOT Benchmark Test

Welcome to the JuPOT Performance Test. As part of the design team's initial requirements we will demonstrate the performance our our platform through a series of timed actions that will normally occur during use


In [1]:
# For Windows Users
push!(LOAD_PATH, "$(homedir())/Documents/RBC_Capstone/financial-opt-tools/src")
using JuPOT
print("") # Dont mind these print statements

<u><b> 1. Measuring the Performance of Optimizing 5000 assets </u></b>

In [2]:
n = 5000 # No. Of Assets
returns = rand(n) # Returns a matrix of size(n) with entries between 0-1s
covariance = let # This part generates a covariance matrix for the returns
    S = randn(n, n)
    S'S + eye(n)
end

tickers = [randstring(3) for i in 1:n] # List of asset names
print("")

<u> Time Taken for Generating Assets Object </u>

In [3]:
# Assets data structure containing, names, expected returns, covarariance
tic();
assets = AssetsCollection(tickers, returns, covariance)
print(toc())
print("")

elapsed time: 1.585520616 seconds
1.585520616

<u> Time Taken for Generating Model Object </u>

In [4]:
tic();
target_return = 0.2
mvo = SimpleMVO(assets, target_return; short_sale=false)
print(toc())
print("")

elapsed time: 0.076049906 seconds
0.076049906

<u> Time Taken for Optimizing the Portfolio </u>

In [5]:
tic();
optimize(mvo)
print(toc())

Optimize a model with 2 rows, 5000 columns and 10000 nonzeros
Model has 12502500 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e-04, 1e+00]
  Objective range [0e+00, 0e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [2e-01, 1e+00]
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve time: 6.86s
Presolved: 2 rows, 5000 columns, 10000 nonzeros
Presolved model has 12502500 quadratic objective terms
Ordering time: 0.24s

Barrier statistics:
 Free vars  : 4999
 AA' NZ     : 1.250e+07
 Factor NZ  : 1.251e+07 (roughly 100 MBytes of memory)
 Factor Ops : 4.170e+10 (roughly 17 seconds per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.76744130e+05 -2.76744130e+05  5.00e+06 1.89e-05  1.00e+06    53s
   1   9.28962171e+04 -9.39730968e+04  3.63e+04 1.37e-07  7.32e+03    90s
   2   4.42458165e+03 -5.51854901e+03  6.65e+02 2.51e-09  1.

<u><b> 2. Measuring the Performance of Optimizing 5000 assets with Constraints</u></b>

<u> Time Taken to Generate Constraints and Parameters Objects </u>

In [12]:
tic();
constraints = [symbol("x$i") => :(min_thresh <= w[$i]) for i=1:n] # this sets a minimum weight for each asset
constraints_2 = [symbol("y$i") => :( w[$i] <= max_thresh) for i=1:n] # this sets a maximum weight for each asset
parameters = Dict(:min_thresh => 0, :max_thresh => 0.7, :n => n)
constraints = merge(constraints,constraints_2) # you just merged three sets of constraints
print(toc())
print("")

elapsed time: 0.053123339 seconds
0.053123339

In [ ]:
tic();
mvo = SimpleMVO(assets, target_return, constraints; short_sale=false)
optimize(mvo, parameters)
print(toc())
print("")